<h2 style='text-align: center;'> Technology Capstone Research Project PG (11522) </h2>
<h3 style='text-align: center;'> Project ID: 2024-S1-50 </h3>
<h3 style='text-align: center;'> Group ID: 11522-24S1-41 </h3>
<h4 style='text-align: center;'> Member: Pauline Armamento - U3246782 </h4>
<h4 style='text-align: center;'> Prompt Technique: Zero-shot </h4>

### HumanEval

The HumanEval dataset released by OpenAI includes 164 programming problems with a function sig- nature, docstring, body, and several unit tests. They were handwritten to ensure not to be included in the training set of code generation models.
https://huggingface.co/datasets/openai_humaneval

### Prompt for Zero-Shot

# Task ID 20
Prompt: Write a Python function that takes a list of numbers and  returns two numbers that are closest to each other in  the order of smallest and largest number.

In [1]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("openai_humaneval")

# Initialize Task Index
task_index = 20
task_153 = dataset["test"][task_index]

# Access Task Details
task_id = task_153["task_id"]
prompt = task_153["prompt"]
canonical_solution = task_153["canonical_solution"]
test_data = task_153["test"]
entry_point = task_153["entry_point"]

# Display Task Details
print(f"Task ID: {task_id}")
print(f"Prompt: {prompt}")
print(f"Canonical Solution: {canonical_solution}")
print(f"Test Data: {test_data}")
print(f"Entry Point: {entry_point}")


Task ID: HumanEval/20
Prompt: from typing import List, Tuple


def find_closest_elements(numbers: List[float]) -> Tuple[float, float]:
    """ From a supplied list of numbers (of length at least two) select and return two that are the closest to each
    other and return them in order (smaller number, larger number).
    >>> find_closest_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.2])
    (2.0, 2.2)
    >>> find_closest_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0])
    (2.0, 2.0)
    """

Canonical Solution:     closest_pair = None
    distance = None

    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                if distance is None:
                    distance = abs(elem - elem2)
                    closest_pair = tuple(sorted([elem, elem2]))
                else:
                    new_distance = abs(elem - elem2)
                    if new_distance < distance:
                        distance = new_distance
                

In [2]:
# Task ID: HumanEval/20
# Function - Zero Shot
# Model 1

import timeit
import pandas as pd

def find_closest_elements1(numbers):
    numbers.sort()
    min_diff = float('inf')
    closest_pair = []

    for i in range(len(numbers) - 1):
        diff = numbers[i + 1] - numbers[i]
        if diff < min_diff:
            min_diff = diff
            closest_pair = [numbers[i], numbers[i + 1]]

    return closest_pair



# Test cases
mod1_output1 = find_closest_elements1([1.0, 2.0, 3.9, 4.0, 5.0, 2.2]) # (3.9, 4.0)
mod1_output2 = find_closest_elements1([1.0, 2.0, 5.9, 4.0, 5.0]) # (5.0, 5.9)
mod1_output3 = find_closest_elements1([1.1, 2.2, 3.1, 4.1, 5.1]) # (2.2, 3.1)


# Execution Time
execution_time_mod1_output1 = timeit.timeit(lambda: find_closest_elements1([1.0, 2.0, 3.9, 4.0, 5.0, 2.2]), number=10000)
print("Execution time Model 1 Output 1:", format(execution_time_mod1_output1, '.5f'))

execution_time_mod1_output2 = timeit.timeit(lambda: find_closest_elements1([1.0, 2.0, 5.9, 4.0, 5.0]), number=10000)
print("Execution time Model 1 Output 2:", format(execution_time_mod1_output2, '.5f'))

execution_time_mod1_output3 = timeit.timeit(lambda: find_closest_elements1([1.1, 2.2, 3.1, 4.1, 5.1]), number=10000)
print("Execution time Model 1 Output 3:", format(execution_time_mod1_output3, '.5f'))

average_time_mod1output = (execution_time_mod1_output1 + execution_time_mod1_output2 + execution_time_mod1_output3) / 3
print("Average Execution Time for Model 1 Output:", format(average_time_mod1output, '.5f'))

def evaluate_list_metrics(expected_output, actual_output):
    if isinstance(expected_output, int) or isinstance(actual_output, int):
        # If either expected_output or actual_output is an integer,
        # convert them to lists for comparison
        expected_output = [expected_output]
        actual_output = [actual_output]

    if len(expected_output) != len(actual_output):
        raise ValueError("Expected and actual output lengths must be the same")

    exact_match_accuracy = 1 if expected_output == actual_output else 0

    true_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual)
    false_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 0 and actual == 1)
    false_negatives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 1 and actual == 0)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual) / len(expected_output)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Exact Match": exact_match_accuracy
    }

expected_outputs = [(3.9, 4.0), (5.0, 5.9), (2.2, 3.1)]
actual_outputs_mod1 = [mod1_output1, mod1_output2, mod1_output3]
print("Expected Output: ", expected_outputs)
print("Actual Output: ", actual_outputs_mod1)

metrics1 = [evaluate_list_metrics(expected, actual) for expected, actual in zip(expected_outputs, actual_outputs_mod1)]

# Create DataFrame directly from the list of dictionaries
df_metrics1_new = pd.DataFrame(metrics1)

# Calculate the average for precision, recall, F1 score, accuracy, and exact match
avg_metrics1 = df_metrics1_new.mean(axis=0)
avg_metrics1.name = 'Model 1 Average'

# Concatenate the average row to the DataFrame
df_metrics1_new = pd.concat([df_metrics1_new, avg_metrics1.to_frame().transpose()])

# Print the new DataFrame
print(df_metrics1_new)



Execution time Model 1 Output 1: 0.01313
Execution time Model 1 Output 2: 0.01076
Execution time Model 1 Output 3: 0.01069
Average Execution Time for Model 1 Output: 0.01153
Expected Output:  [(3.9, 4.0), (5.0, 5.9), (2.2, 3.1)]
Actual Output:  [[3.9, 4.0], [5.0, 5.9], [2.2, 3.1]]
                 Precision  Recall  F1 Score  Accuracy  Exact Match
0                      1.0     1.0       1.0       1.0          0.0
1                      1.0     1.0       1.0       1.0          0.0
2                      1.0     1.0       1.0       1.0          0.0
Model 1 Average        1.0     1.0       1.0       1.0          0.0


In [3]:
# Task ID: HumanEval/95
# Function - Zero Shot
# Model 2


def find_closest_elements2(numbers):
    numbers.sort()
    min_diff = float('inf')
    closest_pair = []

    for i in range(len(numbers) - 1):
        diff = abs(numbers[i + 1] - numbers[i])
        if diff < min_diff:
            min_diff = diff
            closest_pair = [numbers[i], numbers[i + 1]]
        elif diff == min_diff:
            closest_pair.extend([numbers[i], numbers[i + 1]])

    return closest_pair

# Test cases
mod2_output1 = find_closest_elements2([1.0, 2.0, 3.9, 4.0, 5.0, 2.2]) # (3.9, 4.0)
mod2_output2 = find_closest_elements2([1.0, 2.0, 5.9, 4.0, 5.0]) # (5.0, 5.9)
mod2_output3 = find_closest_elements2([1.1, 2.2, 3.1, 4.1, 5.1]) # (2.2, 3.1)


# Execution Time
execution_time_mod2_output1 = timeit.timeit(lambda: find_closest_elements2([1.0, 2.0, 3.9, 4.0, 5.0, 2.2]), number=10000)
print("Execution time Model 2 Output 1:", format(execution_time_mod2_output1, '.5f'))

execution_time_mod2_output2 = timeit.timeit(lambda: find_closest_elements2([1.0, 2.0, 5.9, 4.0, 5.0]), number=10000)
print("Execution time Model 2 Output 2:", format(execution_time_mod2_output2, '.5f'))

execution_time_mod2_output3 = timeit.timeit(lambda: find_closest_elements2([1.1, 2.2, 3.1, 4.1, 5.1]), number=10000)
print("Execution time Model 2 Output 3:", format(execution_time_mod2_output3, '.5f'))

average_time_mod2output = (execution_time_mod2_output1 + execution_time_mod2_output2 + execution_time_mod2_output3) / 3
print("Average Execution Time for Model 2 Output:", format(average_time_mod2output, '.5f'))

def evaluate_list_metrics(expected_output, actual_output):
    if isinstance(expected_output, int) or isinstance(actual_output, int):
        # If either expected_output or actual_output is an integer,
        # convert them to lists for comparison
        expected_output = [expected_output]
        actual_output = [actual_output]

    if len(expected_output) != len(actual_output):
        raise ValueError("Expected and actual output lengths must be the same")

    exact_match_accuracy = 1 if expected_output == actual_output else 0

    true_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual)
    false_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 0 and actual == 1)
    false_negatives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 1 and actual == 0)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual) / len(expected_output)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Exact Match": exact_match_accuracy
    }

expected_outputs = [(3.9, 4.0), (5.0, 5.9), (2.2, 3.1)]
actual_outputs_mod2 = [mod2_output1, mod2_output2, mod2_output3]
print("Expected Output: ", expected_outputs)
print("Actual Output: ", actual_outputs_mod2)

metrics2 = [evaluate_list_metrics(expected, actual) for expected, actual in zip(expected_outputs, actual_outputs_mod2)]

# Create DataFrame directly from the list of dictionaries
df_metrics2_new = pd.DataFrame(metrics2)

# Calculate the average for precision, recall, F1 score, accuracy, and exact match
avg_metrics2 = df_metrics2_new.mean(axis=0)
avg_metrics2.name = 'Model 2 Average'

# Concatenate the average row to the DataFrame
df_metrics2_new = pd.concat([df_metrics2_new, avg_metrics2.to_frame().transpose()])

# Print the new DataFrame
print(df_metrics2_new)



Execution time Model 2 Output 1: 0.01583
Execution time Model 2 Output 2: 0.01504
Execution time Model 2 Output 3: 0.01267
Average Execution Time for Model 2 Output: 0.01451
Expected Output:  [(3.9, 4.0), (5.0, 5.9), (2.2, 3.1)]
Actual Output:  [[3.9, 4.0], [5.0, 5.9], [2.2, 3.1]]
                 Precision  Recall  F1 Score  Accuracy  Exact Match
0                      1.0     1.0       1.0       1.0          0.0
1                      1.0     1.0       1.0       1.0          0.0
2                      1.0     1.0       1.0       1.0          0.0
Model 2 Average        1.0     1.0       1.0       1.0          0.0


In [6]:
# Task ID: HumanEval/20
# Function - Zero Shot
# Model 3

def find_closest_elements3(numbers):
    numbers.sort()
    differences = {}

    # Calculate the differences between adjacent numbers and store in a dictionary
    for i in range(len(numbers) - 1):
        diff = abs(numbers[i + 1] - numbers[i])
        if diff in differences:
            differences[diff].append((numbers[i], numbers[i + 1]))
        else:
            differences[diff] = [(numbers[i], numbers[i + 1])]

    # Find the smallest difference
    min_diff = min(differences.keys())

    # Return the pairs with the smallest difference
    return differences[min_diff]



# Test cases
mod3_output1 = find_closest_elements3([1.0, 2.0, 3.9, 4.0, 5.0, 2.2]) # (3.9, 4.0)
mod3_output2 = find_closest_elements3([1.0, 2.0, 5.9, 4.0, 5.0]) # (5.0, 5.9)
mod3_output3 = find_closest_elements3([1.1, 2.2, 3.1, 4.1, 5.1]) # (2.2, 3.1)


# Execution Time
execution_time_mod3_output1 = timeit.timeit(lambda: find_closest_elements3([1.0, 2.0, 3.9, 4.0, 5.0, 2.2]), number=10000)
print("Execution time Model 3 Output 1:", format(execution_time_mod3_output1, '.5f'))

execution_time_mod3_output2 = timeit.timeit(lambda: find_closest_elements3([1.0, 2.0, 5.9, 4.0, 5.0]), number=10000)
print("Execution time Model 3 Output 2:", format(execution_time_mod3_output2, '.5f'))

execution_time_mod3_output3 = timeit.timeit(lambda: find_closest_elements3([1.1, 2.2, 3.1, 4.1, 5.1]), number=10000)
print("Execution time Model 3 Output 3:", format(execution_time_mod3_output3, '.5f'))

average_time_mod3output = (execution_time_mod3_output1 + execution_time_mod3_output2 + execution_time_mod3_output3) / 3
print("Average Execution Time for Model 3 Output:", format(average_time_mod3output, '.5f'))

def evaluate_list_metrics(expected_output, actual_output):
    # Ensure both expected and actual outputs are lists
    if not isinstance(expected_output, list):
        expected_output = [expected_output]
    if not isinstance(actual_output, list):
        actual_output = [actual_output]

    # Check if lengths are equal
    if len(expected_output) != len(actual_output):
        raise ValueError("Expected and actual output lengths must be the same")

    exact_match_accuracy = 1 if expected_output == actual_output else 0

    true_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual)
    false_positives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 0 and actual == 1)
    false_negatives = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == 1 and actual == 0)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    accuracy = sum(1 for expected, actual in zip(expected_output, actual_output) if expected == actual) / len(expected_output)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Exact Match": exact_match_accuracy
    }

expected_outputs = [(3.9, 4.0), (5.0, 5.9), (2.2, 3.1)]
actual_outputs_mod3 = [mod3_output1, mod3_output2, mod3_output3]
print("Expected Output: ", expected_outputs)
print("Actual Output: ", actual_outputs_mod3)

metrics3 = [evaluate_list_metrics(expected, actual) for expected, actual in zip(expected_outputs, actual_outputs_mod3)]

# Create DataFrame directly from the list of dictionaries
df_metrics3_new = pd.DataFrame(metrics3)

# Calculate the average for precision, recall, F1 score, accuracy, and exact match
avg_metrics3 = df_metrics3_new.mean(axis=0)
avg_metrics3.name = 'Model 3 Average'

# Concatenate the average row to the DataFrame
df_metrics3_new = pd.concat([df_metrics3_new, avg_metrics3.to_frame().transpose()])

# Print the new DataFrame
print(df_metrics3_new)


Execution time Model 3 Output 1: 0.02551
Execution time Model 3 Output 2: 0.01857
Execution time Model 3 Output 3: 0.01962
Average Execution Time for Model 3 Output: 0.02124
Expected Output:  [(3.9, 4.0), (5.0, 5.9), (2.2, 3.1)]
Actual Output:  [[(3.9, 4.0)], [(5.0, 5.9)], [(2.2, 3.1)]]
                 Precision  Recall  F1 Score  Accuracy  Exact Match
0                      1.0     1.0       1.0       1.0          1.0
1                      1.0     1.0       1.0       1.0          1.0
2                      1.0     1.0       1.0       1.0          1.0
Model 3 Average        1.0     1.0       1.0       1.0          1.0


In [7]:
# Task ID: HumanEval/67
# Overall Metrics

# Calculate the averages for each set of metrics
avg_metrics1 = df_metrics1_new.mean(axis=0)
avg_metrics2 = df_metrics2_new.mean(axis=0)
avg_metrics3 = df_metrics3_new.mean(axis=0)

# Calculate the overall average time
overall_average_time = (average_time_mod1output + average_time_mod2output + average_time_mod3output) / 3

# Create a DataFrame to store the averages
df_avg_metrics = pd.DataFrame([avg_metrics1, avg_metrics2, avg_metrics3])

# Add a new column for average time
df_avg_metrics['Average Time'] = [average_time_mod1output, average_time_mod2output, average_time_mod3output]

# Add a new row displaying overall average
df_avg_metrics.loc['Overall Average'] = df_avg_metrics.mean()

# Add a new row for overall average time
df_avg_metrics.loc['Overall Average Time'] = overall_average_time

# Rename the index to include model names
df_avg_metrics.index = ['Model 1', 'Model 2', 'Model 3', 'Overall Average', 'Overall Average Time']

# Print the DataFrame
print(df_avg_metrics)

                      Precision    Recall  F1 Score  Accuracy  Exact Match  \
Model 1                1.000000  1.000000  1.000000  1.000000     0.000000   
Model 2                1.000000  1.000000  1.000000  1.000000     0.000000   
Model 3                1.000000  1.000000  1.000000  1.000000     1.000000   
Overall Average        1.000000  1.000000  1.000000  1.000000     0.333333   
Overall Average Time   0.015759  0.015759  0.015759  0.015759     0.015759   

                      Average Time  
Model 1                   0.011527  
Model 2                   0.014514  
Model 3                   0.021236  
Overall Average           0.015759  
Overall Average Time      0.015759  
